## Crawler Wikipedia

Este notebook se refere a um desenvolvimento de um crawler em python para bucar dados da tabela de informações do wikipedia para páginas de empresas.

![Infobox da KPMG](img\Imagem_1.png)

### Dados importantes

Foi relacionado os seguintes dados como sendo necessários para retorno do Data Frame, excluindo-se os demais campos de informações.

- website oficial

- sede

- pessoas chave 

- subsidiárias

- tipo

- proprietario 

### Busca e retenção da informação

O uso da biblioteca pandas é necessário para a criação do Data Frame porém também utilizamos a função <b>read_hmtl()</b>, que consegue ler a página do wikipedia e retornar a tabela requerida já em um Data Frame, ficando muito mais fácil o futuro tratamento dos dados.

### Os argumentos do pd.read_html:
- header = -1 => necessário pois o infobox de algumas empresas esta formatado de maneira diferente, contendo ou não o logo da empresa, por exemplo https://pt.wikipedia.org/wiki/Alibaba_Group, dessa forma não se corre o risco de deixarmos alguma informação como label das colunas e serem apagadas posteriormente na limpeza dos dados;



- attrs = {'class','infobox_v2'} => Segundo a própria wikipedia (ver: https://pt.wikipedia.org/wiki/Ajuda:Infocaixa ) estas tabelas que existem com um resumo de informações sobre o assunto (no caso, empresas) tem por padrão o valor de classe como 'infobox_v2'. desta forma o valor da classe é passado ao pd.read_html para garantir que a tabela que será lida é a requerida.
          
          
![Classe inspecionada da tabela](img\Imagem_2.png)




A página com a ajuda e padronização da classe inspecionada:

![Pagina de ajuda do wikipedia para a classe infobox_v2](img\Imagem_3.png)

          
### Problemas

- Para páginas em inglês, a classe tem que ser alterada pois a 'infobox_v2' é padronizada para páginas em pt. 

- Algumas páginas, ex: https://pt.wikipedia.org/wiki/PetroChina_Company, não possuem tabelas informativas (infobox_v2), portanto ao tentar ler a tabela o pandas retorna erro.

- Páginas onde a tabela não possui a info "Razão Social", o campo do DataFrame é automaticamente preenchido com o nome presente na url da página do wikipedia, ou seja o endereço após o "wiki/" (https://pt.wikipedia.org/wiki/<b>Alibaba_Group</b>)

### Resultado (Function read_wiki_tables)

A função <b>read_wiki_tables</b> irá receber uma lista com todos os endereços das páginas da url do wikipedia, e caso necessário uma lista com as colunas requeridas - as informações principais estão como default, caso seja necessário mais informações será necessário informar a lista como um todo.

O output é um DataFrame com as colunas preenchidas e uma lista de erros contendo os links das url que não foi possível obter as informações

A célula abaixo faz a importação dos pacotes/módulos que precisamos, no caso o Pandas para a criação dos DataFrames e leitura da página html e o unidecode para a exclusão de sinais especiais (acentos) dos nomes das colunas.

IMPORTANTE:
A lista de input required_col deve ser feita sem estes caracteres especiais, ou seja, sem acentos e em caixa baixa. Como o wikipedia é um site colaborativo, a tabela deve seguir um formato padrão para o html (infobox_v2) porém as informações disponíveis nela e suas disposições são flexíveis, sendo que devemos "fixar" este parâmetro de comparação com o nome dos tipos de informação requeridos à função em caixa baixa e sem acentuação.

In [5]:
import wikipediaapi
import requests
import pandas as pd 
from unidecode import unidecode
import time

In [6]:
wiki_obj=wikipediaapi.Wikipedia('pt')

nome_empresas=pd.read_csv('Base.csv')
nome_empresas.head()

,Empresas
0,kpmg
1,ambev
2,claro
3,vivo
4,alibaba


Criamos uma lista chamada <b>lista_empresas</b> que conterá a coluna <b>Empresas</b> do Data Frame, esta lista necessita de uma limpeza para a execução da busca no API, pois contém informações muito genéricas que em teste prévio com o API não retornaram resultados satisfatórios, ex:

- zf <b>do brasil</b>
- bonzano <b>investimentos</b>
- bosch brasil <b>fundo de investimento em cotas de fundo de investimento multimercado</b>

Estas informações em negrito nos itens acima prejudicam a busca, e muitas vezes retonam resultados nulos. Além disso, haverão muitas informações duplicadas - principalmente após a limpeza das strings - dessa forma é importante excluirmos as strings duplicadas. Estes trechos de string serão excluídos de maneira genérica e manteremos a lista original na list <b>lista_empresas_orig</b>.

In [7]:
lista_empresas=nome_empresas['Empresas']
lista_empresas=list(set(lista_empresas))
lista_empresas_orig=lista_empresas
lista_empresas_orig.sort()
lista_empresas.sort()

In [8]:
lista_empresas=[x.lower().replace('fundo de investimento em acoes','') for x in lista_empresas]
lista_empresas=[x.lower().replace('ltda.','') for x in lista_empresas]
lista_empresas=[x.lower().replace('ltda','') for x in lista_empresas]
lista_empresas=[x.lower().replace('ltd a','') for x in lista_empresas]
lista_empresas=[x.lower().replace('lt da','') for x in lista_empresas]
lista_empresas=[x.lower().replace('ltd a','') for x in lista_empresas]
lista_empresas=[x.lower().replace(' sa ','') for x in lista_empresas]
lista_empresas=[x.lower().replace(' s a','') for x in lista_empresas]
lista_empresas=[x.lower().replace('fundo de investimento','') for x in lista_empresas]
lista_empresas=[x.lower().replace('credito privado','') for x in lista_empresas]
lista_empresas=[x.lower().replace('multimercado','') for x in lista_empresas]
lista_empresas=[x.lower().replace('s.a.','') for x in lista_empresas]
lista_empresas=[x.lower().replace('s/ a','') for x in lista_empresas]
lista_empresas=[x.lower().replace('s/a','') for x in lista_empresas]
lista_empresas=[x.lower().replace('s.a','') for x in lista_empresas]
lista_empresas=[x.lower().replace('s.a','') for x in lista_empresas]
lista_empresas=[x.lower().replace(' sa','') for x in lista_empresas]
lista_empresas=[x.lower().replace(' aluminio','') for x in lista_empresas]
lista_empresas=[x.lower().replace(' limitada','') for x in lista_empresas]
lista_empresas=[x.lower().replace('(brasil)','') for x in lista_empresas]
lista_empresas=[x.lower().replace('(brasil )','') for x in lista_empresas]
lista_empresas=[x.lower().replace(' do brasil','') for x in lista_empresas]
lista_empresas=[x.lower().strip(' ') for x in lista_empresas]

A Function abaixo recebe a lista das empresas, busca a partir da API do wikipedia com a query de "Prefixsearch" que busca o melhor match para a palavra buscada. Com a página(s) identificadas, é possível buscar com ajuda do pacote wikipedia-api se a página linkada possui 'empresas' em sua categoria, isso elimina páginas não relacionadas com o objetivo mas que foram resultados de busca (ex: duplicidade) e retorna a url em caso positivo.

In [9]:
def get_wiki_url(lista):
    
    resultados=pd.DataFrame(columns=['Entity Description','wiki-url','Página não existente'])
    url=''
    erro_url=False
    counter_1=0
    S = requests.Session()
    
    #Pesquisa da página via Wikipedia API
    URL = "https://pt.wikipedia.org/w/api.php"
    
    for item in lista:
        counter_1+=1
        PARAMS = {
            'action':"query",   
            'list':"prefixsearch",
            'pssearch':item,
            'format':"json"
        }
            
        R = S.get(url=URL, params=PARAMS)
        DATA = R.json()
            
            #Verificação se a página existe em pt
        if len(DATA['query']['prefixsearch'])>0:
            titulo=DATA['query']['prefixsearch'][0]['title']
            if len(titulo)<=0:
                erro_url=True
                continue
            #Verificação se a página tem "empresas" nas suas categorias
            if len(wiki_obj.page(title=titulo).categories)>=0:
                categories_dict=wiki_obj.page(title=titulo).categories
                for categ in categories_dict:
                    if 'empresas' in categ.lower():
                        #Obtem a URL da página
                        url=wiki_obj.page(title=titulo).fullurl
                        erro_url=False
                    else:
                        erro_url=True
            else:
                erro_url=True
        else:
            erro_url=True
        
        a={'Entity Description':item,'wiki-url':url,'Página não existente':erro_url}
        resultados.loc[len(resultados)+1]=a
        url,erro_url='',False
        
        if counter_1>=10000:
            resultados.to_csv('save_temporaro.csv')
            print("foram buscadas {} páginas".format(counter_1))
            counter_1=0
        
    return resultados

In [11]:
df_wiki_url=get_wiki_url(lista_empresas)
df_wiki_url

,Entity Description,wiki-url,Página não existente
1,3m,https://pt.wikipedia.org/wiki/3M,True
2,alibaba,https://pt.wikipedia.org/wiki/Alibaba_Group,False
3,ambev,https://pt.wikipedia.org/wiki/AMBEV,True
4,bosch brasil em cotas de,,True
5,bozano investimentos,,True
6,claro,https://pt.wikipedia.org/wiki/Claro,True
7,facebook,https://pt.wikipedia.org/wiki/Facebook,True
8,kpmg,https://pt.wikipedia.org/wiki/KPMG,False
9,microsoft,https://pt.wikipedia.org/wiki/Microsoft,True
10,vivo,https://pt.wikipedia.org/wiki/Vivo,True


Como o resultado para todas as entradas da lista inteira é muito baixo, temos problemas com empresas não existentes no wikipedia, além dos próprios nomes não serem perfeitos para busca. Uma ideia para melhor um pouco o resultado seria buscar no Linkedin. Num pequeno teste, há empresas que não possuem páginas no wiki porém possuem no Linkedin, porém mesmo assim teremos dados faltantes.

Agora será possível com este resultado da function get_wiki_url, inputarmos na function abaixo para obter um dataframe com os dados importantes para o enriquecimento da base.

In [33]:
def read_wiki_tables(url_list,required_col=['razao social','website oficial','sede','pessoas chave',
                                            'subsidiarias','tipo','proprietario'],append_wiki_url=True):
    """Function to read a list of urls from wikipedia and import to a pandas DataFrame
       the infobox of each page.

    Args:
        url_list (list or str): list of urls
        required_col (list): list of data to extract from wikipedia infobox, use lower case and do not use accent 
                             such as ´,`,^,~
        append_wiki_url (Bool): Turn on/off the column wiki_url in DataFrame - used for error analysis

    Returns:
        df (Pandas DataFrame): A DataFrame contaning all informations from infobox section;
        list_of_errors (list): List of wikipedia pages that was impossible to get the infos;
    """   
    
    # inicializa o DataFrame e a lista de erro
    dfinal=pd.DataFrame()
    list_of_errors=[]

    if type(url_list)==str: #verifica se foi enviado o input url como lista ou como string
        url_list=[url_list]
        
    for url in url_list:
        if url=='':
            continue
            
        #faz a leitura da página html e inputa num DataFrame, se erro a url é enviada para a lista e o loop continua
        try:
            dfs=pd.read_html(url,header=None,attrs={'class':'infobox_v2','cellpadding':'3'},flavor='bs4')
        except ValueError:
            list_of_errors.append(url) 
            continue
        
        #O read_html retorna uma lista de DataFrames, portanto é necessário pegar a primeira
        if len(dfs)>=1:               
            df=dfs[0]
            try:
                df.columns=[0,1]
            except ValueError:
                list_of_errors.append(url)
                continue
            
            #executa a limpeza dos dados em função das informações inputadas na lista required_col
            df.dropna(axis=0,how='all',inplace=True)
            df.reset_index(drop=True,inplace=True)           
            df[0]=[unidecode(info_type.lower().strip().replace('-',' ')) for info_type in df[0] if type(info_type)==str]
            
            for idx in range(df.shape[0]):
                if df.loc[idx,0] not in required_col:
                    df.drop(idx,inplace=True)
            
            #Transposição dos dados para que a tabela fique em estrutura correta
            df=df.T
            df.columns=df.loc[0]
            df.drop(0,inplace=True)
            
            #empresas sem o endereço "razão social" tem o final da url inputado na variável para não ficar sem valor
            if 'razao social' not in df.columns:
                df['razao social']=url[30:]
            
            if append_wiki_url:
                df['url']=url

            #monta o dataframe final    
            dfinal=pd.concat([dfinal,df],sort=False)
            
        else:
            list_of_errors.append(url)

    dfinal.reset_index(drop=True,inplace=True)
    
    return dfinal,list_of_errors

Para teste da function, foi levantado uma lista de páginas de empresas do wikipedia a partir dos links abaixo:

- https://pt.wikipedia.org/wiki/Lista_das_maiores_empresas_do_mundo
- https://pt.wikipedia.org/wiki/Lista_dos_maiores_grupos_de_m%C3%ADdia_do_Brasil

Com estas tabelas foi montado o arquiov csv "links_teste_empresas.csv" que contém todas as url disponíveis nas tabelas destas páginas citadas. Esta lista foi inputada na list <b>links</b> que será usada para rodar a <b>function read_wiki_tab

In [25]:
links=df_wiki_url[df_wiki_url["wiki-url"]!=""]["wiki-url"].to_list()
print("Há {} páginas da wiki para entrada na function.".format(len(links)))

Há 8 páginas da wiki para entrada na function.


Para avaliação da função, o tempo para execução da função para 1 url é medido abaixo.

A chamada da function é feita informando o dataframe de retorno, a lista de erros para o retorno e com a lista de endereços de url do wikipedia como argumento para a function de crawler, por ex:

<b>df,erros=read_wiki_tables(links)</b>

In [26]:
links

['https://pt.wikipedia.org/wiki/3M',
 'https://pt.wikipedia.org/wiki/Alibaba_Group',
 'https://pt.wikipedia.org/wiki/AMBEV',
 'https://pt.wikipedia.org/wiki/Claro',
 'https://pt.wikipedia.org/wiki/Facebook',
 'https://pt.wikipedia.org/wiki/KPMG',
 'https://pt.wikipedia.org/wiki/Microsoft',
 'https://pt.wikipedia.org/wiki/Vivo']

In [28]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [34]:
inicio = time.time()
df,erros=read_wiki_tables(links[1])
fim = time.time()
duracao=fim - inicio

print("A duração da function é de " + "%.2f s." % duracao)
df.head()

A duração da function é de 1.09 s.


,tipo,sede,pessoas chave,subsidiarias,website oficial,razao social,url
0,Empresa de capital aberto,"Hangzhou, China","Daniel Zhang,(chairman & CEO) Joseph Tsai,(Vic...","Guangzhou Evergrande Football Club, Taobao, Tm...",[1],Alibaba_Group,https://pt.wikipedia.org/wiki/Alibaba_Group


Com a Function funcionando para uma entrada simples, é possível testar com todo o list <b>links</b> de forma a verificar a funcionalidade da Function para páginas diversas:

### Em caso de incoerência da lista de links com os resutados da function

Para garantir que todos os links inputados na Function foram lidos, é possível criar uma lista auxiliar para informar se algum link deixou de ser registrado. Para isso, habilita-se a opção <b>append_wiki_url</b> de forma que seja criada uma coluna com o endereço da página do wikipedia no DataFrame, e atráves de poucas linhas teremos uma saída se todas algum link deixou de ser inputado na lista de erro ou no DataFrame:

In [37]:
teste_df,erros_teste=read_wiki_tables(links,append_wiki_url=True)
teste_df.head()

,tipo,sede,website oficial,razao social,url,pessoas chave,subsidiarias
0,Empresa de capital aberto,"Maplewood, Minnesota, Estados Unidos",3m.com,3M,https://pt.wikipedia.org/wiki/3M,NaN,NaN
1,Empresa de capital aberto,"Hangzhou, China",[1],Alibaba_Group,https://pt.wikipedia.org/wiki/Alibaba_Group,"Daniel Zhang,(chairman & CEO) Joseph Tsai,(Vic...","Guangzhou Evergrande Football Club, Taobao, Tm..."
2,empresa de capital aberto,"São Paulo, SP, Brasil",www.ambev.com.br,Ambev S.A.,https://pt.wikipedia.org/wiki/AMBEV,"Bernardo Pinto Paiva, diretor geral[1]",Cervecería y Maltería QuilmesCervecería Nacion...
3,Sociedade anônima,"São Paulo, Brasil",claro.com.br,Claro S/A,https://pt.wikipedia.org/wiki/Claro,Paulo Cesar Teixeira,EmbratelNextel Brasil
4,Empresa de capital aberto,"Menlo Park, Califórnia, Estados Unidos",facebook.com,"Meta, Inc.",https://pt.wikipedia.org/wiki/Facebook,Mark Zuckerberg (CEO)Sheryl Sandberg (COO)Davi...,InstagramWhatsAppOculus VR


In [38]:
teste_df.to_csv('Base_de_dados_merge_wikipedia_info.csv')

### Possíveis melhorias

A busca das páginas do wikipedia via API por meio do nome das entidades é o principal problema para enriquecimento desta base. Muitas delas não possuem páginas no wikipedia, outras possuem a página porém sem a tabela de informações e estes problemas reduziram muito os resultados.

A melhor saída seria fazer a busca das mesmas informações via Google ou LinkedIn.

Uma prévia de um possível crawler para o Google Knowledge Panel foi feita, porém sem resultados satisfatórios para uma base grande de dados como a existente (>63k de linhas) pois temos 2 problemas:

- O Google KP API não retorna as informações necessárias pois as informações que requeridas que estão presentes no KP não são oficiais, sendo obtidas por meio de crawling ou infos de usuários, postanto o google nao libera via API estas informações.
- Utilizando "Brute-force" para fazer um crawler para o KP temos problema com o bloqueio de acessos do google e o tempo para executar a função (>10hrs). 

Para criar um crawler do google KP seria necessário mais horas para desenvolver uma function eficiente. 

Além da questão supracitada, seria necessário utilizar algumas horas de desenvolvimento para otimizar as functions acima. Como foram feitas de maneira independentes e anexadas a este notebook elas não foram melhoradas, porém é sabido que num suposto ambiente de produção, este crawler não seria muito eficiente.

Os pontos listados abaixo são melhorias pequenas e que visam aumentar a efetividade da coleta de informações no wikipedia.

É possível analisar páginas em inglês ou outros idiomas para obter um resultado mais rico. Em uma rápida analise de páginas de wikipedia em outros idiomas é possível identificar o mesmo padão de setagens porém com outras classes, como a <b>infobox_vcard</b> para páginas em inglês:

![Inspeção do wiki em ingles](img\Imagem_4.png)


Seria necessário um tratamento mais profundo no DataFrame resultante, pois por exemplo a coluna <b>pessoas chave</b> tem valores nulos (NaN na prévia acima) pois algumas páginas não possuem estas informações, porém possuem informações concomitantes, como Proprietário, fundador e diretores.

![Wiki Folha da Manhã SP](img\Imagem_5.png)


Seria necessário um tratamento na coluna <b>website oficial</b> para padronização das urls, pois algumas possuem o prefixo "www" outras não, assim como um tratamento parecido também seria necessário para coluna <b>subsidiarias</b> pois como é o caso da Folha da Manhã, as suas subsidiárias não estão separadas por nenhum caractere.

Por último, a coluna <b>razao social</b> possui casos como o da Alibaba, que por inexistência deste importante campo de identificação na tabela de informação do wikipedia, a function inputa no campo o fim do endereço da URL do wikipedia, de forma que o valor do campo não necessáriamente é a exata razão social da empresa.

![Exemplo grupo alibaba.png](img\Imagem_6.png)